In [1]:
# ----------------------------
# Cell 1：安裝套件與 UI 設定
# ----------------------------
print("🚀 開始執行 Cell 1：安裝套件與 UI 設定...")

# 1. 基本設定與環境變數
import os
# os.environ["PT_XLA_DEBUG"] = "1" # 可選：開啟 XLA 詳細日誌，除錯時開啟

# 2. 卸載可能衝突的舊版 torch、torch_xla
print("卸載舊版 torch / torch_xla (若存在)...")
!pip -q uninstall -y torch torch_xla torchvision torchaudio torchtext 2>/dev/null || true

# 3. 安裝特定版本的 PyTorch, Torch-XLA (配合 TPU VM)
#    注意：這些版本可能需要根據 Colab 環境更新而調整
#    參考 Colab TPU PyTorch 文件：https://cloud.google.com/tpu/docs/users-guide-pytorch
#    或 PyTorch XLA GitHub：https://github.com/pytorch/xla
print("安裝 PyTorch 與 Torch-XLA (TPU 版本)...")
# 使用 Colab 官方建議的安裝方式 (通常會根據 Colab 環境自動選擇)
# 以下為一個範例，您可能需要根據 Colab 當前推薦版本調整
# !pip install torch~=2.1.0 torch_xla[tpu]~=2.1.0 -f https://storage.googleapis.com/libtpu-releases/index.html
# 為了穩定性，這裡我們沿用您原始筆記本中測試過的版本組合 (若 Colab 環境更新，這可能需要調整)
!pip install -q \
  torch==2.6.0+cpu.cxx11.abi \
  https://storage.googleapis.com/pytorch-xla-releases/wheels/tpuvm/torch_xla-2.6.0%2Bcxx11-cp311-cp311-manylinux_2_28_x86_64.whl \
  "torch_xla[tpu]==2.6.0" \
  -f https://storage.googleapis.com/libtpu-releases/index.html \
  -f https://storage.googleapis.com/libtpu-wheels/index.html \
  -f https://download.pytorch.org/whl/torch

# 4. 安裝 Hugging Face Transformers 與其他必要套件
print("安裝 Hugging Face Transformers 及其他輔助套件...")
# 鎖定 transformers 版本以確保 API 穩定性，可依需求更新
!pip install -q "transformers>=4.39.0,<4.40.0" sentencepiece librosa soundfile ipywidgets

# 5. 安裝系統音訊解碼工具 ffmpeg
print("更新 apt 並安裝 ffmpeg...")
!apt update -qq && apt install -y -qq ffmpeg

print("\n✅ 套件安裝完成！")

# 6. 匯入 ipywidgets 並建立設定介面
import ipywidgets as widgets
from IPython.display import display, HTML

print("\n⚙️ 請進行以下轉錄設定：")

# 增加一些 CSS 來美化 widgets
display(HTML("""
<style>
    .widget-label { min-width: 20ex !important; }
    .widget-dropdown > select { background-color: #f0f0f0; }
    .widget-text input[type="text"] { background-color: #f0f0f0; }
    .widget-button { background-color: #4CAF50 !important; color: white !important; }
</style>
"""))

model_options = [
    ("Tiny (最快, 準確度較低)", "tiny"),
    ("Base (快速, 基礎準確度)", "base"),
    ("Small (推薦, 速度與準確度均衡)", "small"),
    ("Medium (較慢, 準確度高)", "medium"),
    ("Large-v2 (最慢, 準確度最高)", "large-v2"),
    ("Large-v3 (最新, 準確度可能更高)", "large-v3") # large-v3 可能需要更新的 transformers
]
model_widget = widgets.Dropdown(options=model_options, value="small", description="Whisper 模型:")

language_widget = widgets.Text(value="zh", description="轉錄語言:", placeholder="例如: zh, en, ja, auto")
task_widget = widgets.Dropdown(options=["transcribe", "translate"], value="transcribe", description="任務:")

precision_options = [
    ("FP32 (標準精度)", "fp32"),
    ("BF16 (TPU 加速, 略降精度)", "bf16")
]
precision_widget = widgets.Dropdown(options=precision_options, value="bf16", description="運算精度:")

# 長音檔處理參數 (可讓進階使用者調整)
chunk_length_s_widget = widgets.IntText(value=28, description="音訊切塊長度(秒):", style={'description_width': 'initial'}) # 預設 28 秒，略小於30秒以保留邊界
stride_length_s_left_widget = widgets.IntText(value=5, description="左側重疊(秒):", style={'description_width': 'initial'})
stride_length_s_right_widget = widgets.IntText(value=5, description="右側重疊(秒):", style={'description_width': 'initial'})

# 建立一個 Box 來組織 widgets
settings_box = widgets.VBox([
    model_widget,
    language_widget,
    task_widget,
    precision_widget,
    widgets.HTML("<b>長音檔處理 (進階設定):</b>"),
    chunk_length_s_widget,
    stride_length_s_left_widget,
    stride_length_s_right_widget
])

display(settings_box)

print("\n Cell 1 設定完成。請確認以上設定，然後執行下一個 Cell。")

🚀 開始執行 Cell 1：安裝套件與 UI 設定...
卸載舊版 torch / torch_xla (若存在)...
安裝 PyTorch 與 Torch-XLA (TPU 版本)...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 MB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
安裝 Hugging Face Transformers 及其他輔助套件...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 135.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 83.1 MB/s eta 0:00:00
更新 apt 並安裝 ffmpeg...
6 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources


 Cell 1 設定完成。請確認以上設定，然後執行下一個 Cell。


In [2]:
# -------------------------------------------------
# Cell 2：載入模型、初始化 Pipeline 與 XLA 熱機
# -------------------------------------------------
print("🚀 開始執行 Cell 2：載入模型、初始化 Pipeline 與 XLA 熱機...")

# 1. 匯入必要函式庫
import torch
import torch_xla
import torch_xla.core.xla_model as xm
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
import numpy as np
import time
import warnings

# 靜音一些不影響功能的警告
warnings.filterwarnings("ignore", message=".*TorchScript only supports basic types list, tuple, dict.*")
warnings.filterwarnings("ignore", message=".*PySoundFile failed.*") # m4a/mp3 → audioread fallback
warnings.filterwarnings("ignore", message=".*Due to a bug fix.*")   # HF Transformers 多語行為提示

# 2. 取得使用者在 Cell 1 的設定
selected_model_name_suffix = model_widget.value
selected_language = language_widget.value
selected_task = task_widget.value
selected_precision = precision_widget.value
chunk_length = chunk_length_s_widget.value
stride_left = stride_length_s_left_widget.value
stride_right = stride_length_s_right_widget.value

MODEL_NAME = f"openai/whisper-{selected_model_name_suffix}"

# 3. 設定 TPU 裝置與運算精度
if 'xm' not in globals(): # 確保 xm 已被正確匯入
    import torch_xla.core.xla_model as xm
    print("torch_xla.core.xla_model (xm) 已重新匯入。")

try:
    device = xm.xla_device()
    print(f"✅ TPU 裝置成功取得: {device}")
    print(f"檢測到 XLA 裝置: {xm.get_xla_supported_devices()}")
except Exception as e:
    print(f"⚠️ 無法取得 TPU 裝置，錯誤: {e}")
    print("請確認 Colab Runtime 已選擇 TPU。後續處理將可能失敗。")
    device = torch.device("cpu") # Fallback to CPU if TPU not found


if selected_precision == "bf16":
    torch_dtype = torch.bfloat16
    print("運算精度設定為: BF16")
else:
    torch_dtype = torch.float32
    print("運算精度設定為: FP32")

# 4. 載入 Whisper Processor
print(f"\n載入 Whisper Processor for {MODEL_NAME}...")
try:
    processor = WhisperProcessor.from_pretrained(MODEL_NAME)
    print("✅ Processor 載入成功!")
except Exception as e:
    print(f"❌ Processor 載入失敗: {e}")
    processor = None # 標記失敗

# 5. 載入 Whisper 模型並移至 TPU
if processor: # 只有 processor 成功載入才繼續
    print(f"載入 Whisper 模型 {MODEL_NAME} 並移至 {device} (dtype: {torch_dtype})...")
    try:
        model = WhisperForConditionalGeneration.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch_dtype # 在載入時就指定精度
        ).to(device)
        model.eval() # 設定為評估模式
        print("✅ 模型載入並移至 TPU 成功!")
        print(f"模型最大目標位置 (max_target_positions): {model.config.max_target_positions}")
    except Exception as e:
        print(f"❌ 模型載入或移至 TPU 失敗: {e}")
        model = None # 標記失敗
else:
    model = None
    print("由於 Processor 載入失敗，跳過模型載入。")


# 6. 初始化 ASR Pipeline
asr_pipeline = None # 先給定初始值
if model and processor: # 只有模型和 processor 都成功載入才繼續
    print("\n初始化 ASR Pipeline...")
    try:
        # 注意：device 參數在 pipeline 中若模型已在特定設備上，會自動偵測。
        # torch_dtype 也會從模型繼承。
        asr_pipeline = pipeline(
            "automatic-speech-recognition",
            model=model, # 直接傳入已載入並移至 TPU 的模型
            tokenizer=processor.tokenizer,
            feature_extractor=processor.feature_extractor,
            torch_dtype=torch_dtype, # 再次確認 dtype
            device=device # 明確指定 device 給 pipeline
        )
        print("✅ ASR Pipeline 初始化成功!")
    except Exception as e:
        print(f"❌ ASR Pipeline 初始化失敗: {e}")
else:
    print("由於模型或 Processor 載入失敗，跳過 Pipeline 初始化。")

# 7. XLA 熱機 (Warm-up)
#    透過處理一小段靜音來觸發 XLA 編譯
if asr_pipeline:
    print("\n🔥 開始 XLA 熱機 (處理 2 秒靜音)...")
    # 產生 2 秒靜音, 16kHz
    dummy_audio = np.zeros(16000 * 2, dtype=np.float32)

    # 設定 generate_kwargs，語言和任務從 widget 取得
    # 對於 "auto" 語言偵測，generate_kwargs 中不需指定 language
    generate_pipeline_kwargs = {"task": selected_task}
    if selected_language.lower() != "auto":
        generate_pipeline_kwargs["language"] = selected_language

    # Hugging Face pipeline 會自動處理 prompt ID。
    # 如果遇到特定語言需要更精確的 prompt ID 控制，可以如下設定：
    # prompt_ids = processor.get_decoder_prompt_ids(language=selected_language, task=selected_task)
    # generate_pipeline_kwargs["prompt_ids"] = prompt_ids

    t_start_warmup = time.time()
    try:
        with torch.no_grad(): # 推理時不需要梯度
            _ = asr_pipeline(
                dummy_audio,
                generate_kwargs=generate_pipeline_kwargs,
                chunk_length_s=chunk_length, # 使用者設定的切塊長度
                stride_length_s=[stride_left, stride_right] # 使用者設定的重疊長度
            )
        xm.mark_step() # 確保 XLA 操作完成
        t_elapsed_warmup = time.time() - t_start_warmup
        print(f"✅ XLA 熱機完成，耗時 {t_elapsed_warmup:.2f} 秒。")
        print("現在可以執行 Cell 3 來處理您的音檔了。")
    except Exception as e:
        print(f"❌ XLA 熱機失敗: {e}")
        print("後續轉錄可能緩慢或失敗。請檢查錯誤訊息。")
else:
    print("\n⚠️ Pipeline 未成功初始化，跳過 XLA 熱機。請檢查 Cell 1 和 Cell 2 的錯誤訊息。")

# 清理不再需要的變數以釋放記憶體 (可選)
# import gc
# del dummy_audio
# gc.collect()

🚀 開始執行 Cell 2：載入模型、初始化 Pipeline 與 XLA 熱機...
✅ TPU 裝置成功取得: xla:0
檢測到 XLA 裝置: ['xla:0']
運算精度設定為: BF16

載入 Whisper Processor for openai/whisper-small...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

✅ 模型載入並移至 TPU 成功!
模型最大目標位置 (max_target_positions): 448

初始化 ASR Pipeline...
✅ ASR Pipeline 初始化成功!

🔥 開始 XLA 熱機 (處理 2 秒靜音)...
✅ XLA 熱機完成，耗時 86.13 秒。
現在可以執行 Cell 3 來處理您的音檔了。


In [4]:
# --------------------------------------------
# Cell 3：上傳音檔並進行長音檔轉錄
# --------------------------------------------
print("🚀 開始執行 Cell 3：上傳音檔並進行長音檔轉錄...")

# 1. 匯入必要模組
from google.colab import files
import librosa
import time
import os

# 2. 檢查 Pipeline 是否已準備就緒
if 'asr_pipeline' not in globals() or asr_pipeline is None:
    print("❌ ASR Pipeline 尚未初始化或初始化失敗。請先成功執行 Cell 1 和 Cell 2。")
    # 你可以在這裡決定是否要 raise Exception 來停止執行
    # raise RuntimeError("ASR Pipeline not ready. Please run previous cells successfully.")
else:
    print("✅ ASR Pipeline 已準備就緒。")

    # 3. 提示使用者上傳音檔
    print("\n📤 請上傳音檔 (mp3 / wav / m4a / ogg / flac ...)")
    print("   您可以一次選擇多個檔案。")
    try:
        uploaded_files = files.upload() # Colab 檔案上傳介面
        if not uploaded_files:
            print(" 🤔 沒有上傳任何檔案。")
        else:
            print(f"📂 已上傳 {len(uploaded_files)} 個檔案。")
    except Exception as e:
        print(f"❌ 檔案上傳過程中發生錯誤: {e}")
        uploaded_files = {} # 確保 uploaded_files 是個字典以避免後續錯誤

    # 4. 定義日誌函式
    def show_log(msg):
        print(time.strftime("[%H:%M:%S]"), msg)

    # 5. 逐個檔案處理
    if uploaded_files: # 確保有檔案被上傳
        # 從 Cell 2 widgets 中再次獲取最新的語言和任務設定 (以防使用者在 Cell 2 執行後又改了 Cell 1 的 widget)
        current_language = language_widget.value
        current_task = task_widget.value
        current_chunk_length = chunk_length_s_widget.value
        current_stride_left = stride_length_s_left_widget.value
        current_stride_right = stride_length_s_right_widget.value

        # 準備 pipeline 的 generate_kwargs
        # 對於 "auto" 語言偵測，generate_kwargs 中不需指定 language
        generate_pipeline_kwargs_main = {"task": current_task}
        if current_language.lower() != "auto":
            generate_pipeline_kwargs_main["language"] = current_language

        show_log(f"📝 開始處理 {len(uploaded_files)} 個音檔...")
        show_log(f"   語言設定: {current_language}, 任務: {current_task}")
        show_log(f"   切塊長度: {current_chunk_length}s, 左右重疊: {current_stride_left}s / {current_stride_right}s")


        for fname_original, file_content in uploaded_files.items():
            show_log(f"--- 開始處理檔案: {fname_original} ---")

            # 將上傳的內容寫入暫存檔案，因為 librosa.load 和 pipeline 通常需要檔案路徑
            # 或者，pipeline 有些可以直接接受 bytes，但路徑更通用
            temp_audio_path = f"./{fname_original}" # 直接存在 Colab 當前目錄
            with open(temp_audio_path, "wb") as f:
                f.write(file_content)

            try:
                # (可選步驟) 使用 librosa 獲取音訊時長資訊 (非必要，pipeline會自己處理)
                # wav, sr = librosa.load(temp_audio_path, sr=16000)
                # duration_seconds = librosa.get_duration(y=wav, sr=sr)
                # show_log(f"  音訊長度約: {duration_seconds:.2f} 秒 (取樣率: {sr} Hz)")
                # del wav # 釋放記憶體

                # 核心轉錄步驟
                show_log(f"  🤖 使用 ASR Pipeline 進行轉錄 (這可能需要一些時間，依音檔長度而定)...")
                t_transcribe_start = time.time()

                # 使用 asr_pipeline 進行轉錄
                # pipeline 可以直接吃檔案路徑
                with torch.no_grad(): # 推理時不需要梯度
                    output = asr_pipeline(
                        temp_audio_path, # 直接傳遞檔案路徑
                        chunk_length_s=current_chunk_length,
                        stride_length_s=[current_stride_left, current_stride_right],
                        generate_kwargs=generate_pipeline_kwargs_main,
                        return_timestamps=False # 設為 True 可以得到帶時間戳的更詳細結果，但 text 會變成分段列表
                    )

                xm.mark_step() # 確保 XLA 操作完成
                t_transcribe_elapsed = time.time() - t_transcribe_start

                transcription_text = output["text"]
                show_log(f"  ✅ 轉錄完成，耗時 {t_transcribe_elapsed:.2f} 秒。")

                # 輸出預覽
                preview_length = 200
                preview = transcription_text[:preview_length] + ("..." if len(transcription_text) > preview_length else "")
                print(f"\n📜 轉錄結果預覽 (前 {preview_length} 字元):\n\"{preview}\"")

                # 儲存完整逐字稿
                base_fname, _ = os.path.splitext(fname_original)
                out_filename = f"{base_fname}_transcript_{selected_model_name_suffix}_{current_language}.txt"
                with open(out_filename, "w", encoding="utf-8") as f:
                    f.write(transcription_text)
                show_log(f"  💾 完整逐字稿已儲存至: {out_filename}")

            except Exception as e:
                show_log(f"  ❌ 處理檔案 {fname_original} 時發生錯誤: {e}")
                # 可以考慮在這裡印出更詳細的 traceback
                # import traceback
                # traceback.print_exc()
            finally:
                # 清理暫存檔案
                if os.path.exists(temp_audio_path):
                    os.remove(temp_audio_path)
                show_log(f"--- 檔案 {fname_original} 處理結束 ---\n")

        show_log("🎉🎉🎉 所有音檔處理完畢！請至左側「檔案」面板下載 *_transcript.txt 檔案。🎉🎉🎉")

🚀 開始執行 Cell 3：上傳音檔並進行長音檔轉錄...
✅ ASR Pipeline 已準備就緒。

📤 請上傳音檔 (mp3 / wav / m4a / ogg / flac ...)
   您可以一次選擇多個檔案。


Saving love.m4a to love.m4a
📂 已上傳 1 個檔案。
[02:42:05] 📝 開始處理 1 個音檔...
[02:42:05]    語言設定: zh, 任務: transcribe
[02:42:05]    切塊長度: 28s, 左右重疊: 5s / 5s
[02:42:05] --- 開始處理檔案: love.m4a ---
[02:42:05]   🤖 使用 ASR Pipeline 進行轉錄 (這可能需要一些時間，依音檔長度而定)...
[02:46:15]   ✅ 轉錄完成，耗時 250.01 秒。

📜 轉錄結果預覽 (前 200 字元):
"有你的心跳,但女生永遠不愛你,永遠不會"
[02:46:15]   💾 完整逐字稿已儲存至: love_transcript_small_zh.txt
[02:46:15] --- 檔案 love.m4a 處理結束 ---

[02:46:15] 🎉🎉🎉 所有音檔處理完畢！請至左側「檔案」面板下載 *_transcript.txt 檔案。🎉🎉🎉
